In [ ]:
import psana as ps
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from lv17analysis import epix, lv17data, detectors

run = [457]
ds = ps.DataSource(exp=lv17data.exp, run=run)

ds_run = next(ds.runs())

photon_energy_kev = lv17data.photon_energy_kev(ds_run.runnum)
hit_thresh = 0
lit_thresh = 0.75 * photon_energy_kev
nlit_thresh = 30000
cmap = 'magma'
lognorm = LogNorm(vmin=photon_energy_kev/2, vmax=100*photon_energy_kev)
mask = np.array([])

fig = plt.figure(figsize=(40, 20))

while True:
    evt = next(ds_run.events())
    
    photon_energy = detectors.get_photon_energy(ds_run, evt)
    gmd = detectors.get_gmd(ds_run, evt)
    xgmd = detectors.get_xgmd(ds_run, evt)
    source_on = detectors.get_source_on(ds_run, evt)
    img = epix.get_img(ds_run, evt, masked=False)

    if gmd is None:
        continue

    if xgmd is None:
        continue

    if img is None:
        continue

    if not source_on:
        continue

    img_masked = epix.masked_img(img, mask=mask)
    mask = np.ma.getmask(img_masked)

    lit_pix = epix.count_lit(img, lit_thresh=lit_thresh)

    # plt.pcolormesh(epix_img, cmap=cmap, norm=LogNorm())
    # break

    epix_sum = np.nansum(img.flatten())
    if epix_sum > hit_thresh and lit_pix > nlit_thresh:
        fig.add_subplot(1,2,1)
        plt.pcolormesh(img, cmap=cmap, norm=lognorm)
        plt.title('sum={:.3g}, lit={:.3g}'.format(epix_sum, lit_pix))
        plt.axis('image')
        # plt.tight_layout()
        plt.colorbar()
        
        fig.add_subplot(1,2,2)
        plt.pcolormesh(img_masked, cmap=cmap, norm=lognorm)
        plt.axis('image')
        # plt.tight_layout()
        plt.colorbar()
        # plt.pcolormesh(epix_img, cmap=cmap, norm=LogNorm())
        
        # plt.title('sum={:.3g}, lit={:.3g}, timing={:.3g}'.format(epix_sum, lit_pix, timing_data))
        break